In [10]:
import pandas as pd
import numpy as np
from collections import Counter,defaultdict

df = pd.read_csv('./formatted_ufo_reports.csv',names=['Date-obs','Day','Month','Year','Time','Date_Posted',
                                                   'Day_Posted','Month_Posted','Year_Posted','City','State','Country',
                      'Shape','Time_mins','Time_text','Experiences','Lattitude','Longitude']);
df.head(5)

,,,Date-obs,Day,Month,Year,Time,Date_Posted,Day_Posted,Month_Posted,Year_Posted,City,State,Country,Shape,Time_mins,Time_text,Experiences,Lattitude,Longitude
NaN,level_0,index,Date_obs,Day,Month,Year,Time,Date_Posted,Day_Posted,Month_Posted,Year_Posted,City,State,Country,Shape,Time_mins,Time_text,Experiences,Lattitude,Longitude
0.0,0,0,10/10/1949,10,10,1949,20:30,4/27/2004,27,4,2004,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,29.8830556,-97.9411111
1.0,1,1,10/10/1949,10,10,1949,21:00,12/16/2005,16,12,2005,lackland afb,tx,us,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,29.38421,-98.581082
3.0,3,3,10/10/1956,10,10,1956,21:00,1/17/2004,17,1,2004,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,28.9783333,-96.6458333
4.0,4,4,10/10/1960,10,10,1960,20:00,1/22/2004,22,1,2004,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,21.4180556,-157.8036111


In [18]:

df_sent = pd.read_csv('./formatted_with_sentiment.csv');
df_sent.head(2)

def convert_to_hours(time_min):
    return np.floor(time_min/3600)+1

time_hrs=map(convert_to_hours,df_sent.Time_mins)
#print time_hrs
df_sent['Time_hrs']=time_hrs
df_sent.to_csv('./formatted_with_sentiment_thrs.csv')

In [5]:
years=np.unique(df.Year.values)
countries=np.unique(df.Country.values) 
collective= df.groupby(['Year','Country']);

list_of_subdfs=[];

print countries
for year in years:
    for country in countries:
        if(country!="Country"):
            #print (year,country)
            try:
                filtered=collective.get_group((str(year),str(country)))
               # print len(filtered)
                if(len(filtered)>=10):
                    sub_df=filtered.iloc[np.random.choice(len(filtered),10),:]
                    list_of_subdfs.append(sub_df);
                else:
                    list_of_subdfs.append(filtered)
            except:
                pass
             #   print("Not Found")
                
final_df=pd.concat(list_of_subdfs)
#print final_df
print(len(final_df))
final_df.to_csv("./year_country_filtered.csv")

['Country' 'au' 'ca' 'gb' 'us']
7749


In [4]:
years_list=list(np.unique(df.Year.values))
all_years=df.Year.values
print(len(years_list))
print(len(all_years))

year_counts=dict(df.Year.value_counts())

years=year_counts.keys()
counts=year_counts.values()

year_country_counts=[];
year_country_max=[];

for year in years_list:
    year_df=df[df.Year==year]
    #print year
    country_counts=dict(year_df.Country.value_counts())
    year_country_counts.append(country_counts)
    
year_=[]
countries=np.unique(df.Country.values)
countries_=[[] for _ in range(len(countries))]
max_country=[];
max_value=[];

for idx,year in enumerate(years):
    country_stats=year_country_counts[idx]
    year_.append(year) 
    max_country.append(max(country_stats,key=country_stats.get))
    max_value.append(country_stats[max(country_stats,key=country_stats.get)])
    for i,country_name in enumerate(countries):
        try:
            countries_[i].append(country_stats[country_name])
        except:
            countries_[i].append(0)
                
print len(countries_)            

#month-wise stats each year
year_month_counts=[];
months=range(1,13,1);
months_=[[] for _ in range(1,13,1)];

for year in years_list:
    year_month_counts.append(dict(df[df.Year==year].Month.value_counts()))

for idx,year in enumerate(years):   
    month_stats=year_month_counts[idx]
    for i,m in enumerate(months):
        try:
            months_[i].append(month_stats[m])
           
        except:
            months_[i].append(0)
   
        
new_df=pd.DataFrame({'year':year_,'1':months_[0],
        '2':months_[1],
        '3':months_[2],
        '4':months_[3],
        '5':months_[4],
        '6':months_[5],
         '7':months_[6],
         '8':months_[7],
         '9':months_[8],
         '10':months_[9],
         '11':months_[10],
          '12':months_[11],           
        'Country':countries_[0],'au':countries_[1],'ca':countries_[2],'gb':countries_[3],
        'us':countries_[4],'max_country':max_country,'max_value':max_value})

new_df.head(5)
new_df.to_csv('./year_month_country.csv')

161
72934
5


In [6]:
shapes=np.unique(df.Shape.values)
year_shape_stats=[];

shapes_=[[] for _ in range(len(shapes))]

for year in years:
    year_df=df[df.Year==year]
    year_shape_stats.append(year_df.Shape.value_counts())

print shapes   
for idx,year in enumerate(years):   
    shape_stats=year_shape_stats[idx]
    for i,s in enumerate(shapes):
        try:
            shapes_[i].append(shape_stats[s])
           
        except:
            shapes_[i].append(0)
            
new_df2=pd.DataFrame({'year':year_,
        'Shape':shapes_[0],
        'changed':shapes_[1],
        'changing':shapes_[2],
        'chevron':shapes_[3],
        'cigar':shapes_[4],
        'circle':shapes_[5],
         'cone':shapes_[6],
         'crescent':shapes_[7],
         'cross':shapes_[8],
         'cylinder':shapes_[9],
         'delta':shapes_[10],
          'diamond':shapes_[11], 
          'disk':shapes_[12],
           'dome':shapes_[13],
            'egg':shapes_[14],
            'fireball':shapes_[15],
            'flare':shapes_[16],
            'formation':shapes_[17],
            'hexagon':shapes_[18],
            'light':shapes_[19],
            'other':shapes_[20],
            'oval':shapes_[21],
            'pyramid':shapes_[22],
            'rectangle':shapes_[23],
            'round':shapes_[24],
            'sphere':shapes_[25],
             'teardrop':shapes_[26],
             'triangle':shapes_[27],
              'unknown':shapes_[28]})

new_df2.head(5)
new_df2.to_csv('./year_shape_country.csv')

['Shape' 'changed' 'changing' 'chevron' 'cigar' 'circle' 'cone' 'crescent'
 'cross' 'cylinder' 'delta' 'diamond' 'disk' 'dome' 'egg' 'fireball'
 'flare' 'flash' 'formation' 'hexagon' 'light' 'other' 'oval' 'pyramid'
 'rectangle' 'round' 'sphere' 'teardrop' 'triangle' 'unknown']


In [7]:
year_=[];
month_=[];
cities=np.unique(df.City.values)

city_=[[] for _ in range(len(cities))];
shapes_=[[] for _ in range(len(shapes))];

for year in years:
    for month in range(1,13,1):
        try:
            city_stats= df.groupby(['Year','Month']).get_group((str(year),str(month))).City.value_counts()
            year_.append(year)
            month_.append(month)      
            for i,c in enumerate(cities):
                try:
                    city_[i].append(city_stats[c])
                except:
                    city_[i].append(0)
        except:
            pass
        
print cities

new_df3=pd.DataFrame({'Year':year_,'Month':month_})
for i,city in enumerate(cities):
    new_df3[city]=city_[i]
    
new_df3.head(3) 
new_df3.to_csv('./year_month_city.csv')

['1-25 corridor (southbound&#44 65 miles north nm border)'
 '100 mile (canada)' '100 mile house (canada)' ..., 'zortman' 'zumbrota'
 'zwolle']


In [11]:

year_=[];
month_=[];
cities=np.unique(df.City.values)

city_=[[] for _ in range(len(cities))];
shapes_=[[] for _ in range(len(shapes))];

for year in years:
    for month in range(1,13,1):
        try:
            shape_stats= df.groupby(['Year','Month']).get_group((str(year),str(month))).Shape.value_counts()
            year_.append(year)
            month_.append(month)      
            for i,c in enumerate(shapes):
                try:
                    shapes_[i].append(shape_stats[c])
                except:
                    shapes_[i].append(0)
        except:
            pass
        
print shapes

new_df4=pd.DataFrame({'Year':year_,'Month':month_})
for i,shape in enumerate(shapes):
    new_df4[shape]=shapes_[i]
    
new_df4.head(3) 
new_df4.to_csv('./year_month_shape.csv')


['Shape' 'changed' 'changing' 'chevron' 'cigar' 'circle' 'cone' 'crescent'
 'cross' 'cylinder' 'delta' 'diamond' 'disk' 'dome' 'egg' 'fireball'
 'flare' 'flash' 'formation' 'hexagon' 'light' 'other' 'oval' 'pyramid'
 'rectangle' 'round' 'sphere' 'teardrop' 'triangle' 'unknown']
